# Water Level Data

### Code Summary:

- Detrending, normalizing, and shifting data to zero mean
- Fast Fourier transform magnitude and phase spectra plots
- Continuous wavelet transform scalograms with Morlet wavelet
- Subsampled data plots
- Tapered data plots
- Histograms and statistics for plots

In [ ]:
# Data source: https://www.isdm-gdsi.gc.ca/isdm-gdsi/twl-mne/maps-cartes/inventory-inventaire-eng.asp?user=isdm-gdsi&region=MEDS&tst=1&perm=0

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
import scipy.stats as ss
from sklearn.preprocessing import MinMaxScaler
import scipy.fft
import pywt
import plotly.graph_objects as go
import plotly.tools as tls
from plotly.subplots import make_subplots
from scipy.signal.windows import hann
from scipy import interpolate
from scipy.ndimage import uniform_filter

In [ ]:
df1 = pd.read_csv("Toronto-daily-mean-water-1960-2024.csv", index_col="Obs_date")

# Check for null values in dataframe:
any_null = df1.isnull().any().any()
print(any_null) # Will show false if no null values

In [ ]:
df1.head()

In [ ]:
# Linearly detrend data, normalize and shift to zero mean:
df1['Detrended'] = signal.detrend(df1['SLEV(metres)'], type='linear')

scaler = MinMaxScaler()
df1['Normalized'] = scaler.fit_transform(df1['Detrended'].values.reshape(-1, 1)).flatten()
print(f"Min-Max normalized range: [{df1['Normalized'].min():.3f}, {df1['Normalized'].max():.3f}]")

# Shift Mean:
df1['Normalized_Shifted'] = (df1['Normalized'] - 0.5)

***
### Fast Fourier Transform (Subsampled and Tapered Data)
#### Notes:
- Applying the fast fourier transform to subsampled and tapered DNS data to plot the resulting magnitude and phase spectra
<br> Effects of subsampling/tapering:
    - removes higher frequency content
    - reduces spectral leakage caused by discontinuities
<br>
- Change plot x-limits to zoom in on points of interest

In [ ]:
# From previous subsampling and tapering related code, determined that step = 52 goes well with percent_check,
# and taper with no padding prevents discontinuities
sub_step = 52

L = len(df1['Normalized_Shifted'])
data = df1['Normalized_Shifted'].values
time_index = pd.to_datetime(df1.index)
# time delta (1 day for water level data)
time_delta = (time_index[1] - time_index[0])

window = hann(L)
tapered_sub = (window * data)[::sub_step]

tapered_subsampled = pd.DataFrame(tapered_sub)
tapered_subsampled['date_column'] = df1.index[::sub_step] # Add date column
tapered_subsampled.rename(columns={0: 'Subsampled_Tapered'}, inplace=True)
tapered_subsampled.head()

In [ ]:
# FFT plot display
# (plotly wasn't working here)
def apply_hann_window(data):
    """Apply Hann window and return windowed data with coherent gain factor."""
    window = np.hanning(len(data))
    windowed_data = data * window
    # Coherent gain for Hann window is 0.5, so need to multiply by 2 later on to make correction
    coherent_gain = 0.5
    return windowed_data, coherent_gain


def subsample_with_antialiasing(data, decimation_factor, order, sr=1):
    """Subsample data using scipy.signal.decimate with anti-aliasing filter.
    data: array
    decimation_factor: factor by which to reduce sampling rate
    sr: Original sampling rate (1 sample/day)
    order: order of the filter
    """
    # scipy.signal.decimate applies an anti-aliasing filter automatically
    # Use a higher order filter for better anti-aliasing (default is 8)
    decimated_data = scipy.signal.decimate(data, decimation_factor, order, ftype='iir', zero_phase=True)
    # get new sampling rate:
    new_sr = sr / decimation_factor
    return decimated_data, new_sr


def plotly_fft_2(data, sr, fig1=None, fig2=None, fig3=None, name='', apply_window=True):
    """Plot FFT with windowing correction.
    data: array
    sr: Sampling rate
    apply_window : bool, says whether to apply Hann window (default True)
    """
    n = len(data)
    
    # Apply window if requested
    if apply_window:
        windowed_data, coherent_gain = apply_hann_window(data)
    else:
        windowed_data = data
        coherent_gain = 1.0
    
    # Compute FFT
    fft_values = scipy.fft.rfft(windowed_data)
    freqs = scipy.fft.rfftfreq(n, d=1/sr)  # d = 1/sr because freqs should be in cycles/day
    freqs_per_year = freqs * 365.25
    
    # Apply coherent gain correction to magnitudes
    magnitudes = np.abs(fft_values) * (2.0/n) / coherent_gain
    
    # DC component and Nyquist (if present) should not be doubled
    magnitudes[0] /= 2.0
    if n % 2 == 0:  # If even length, Nyquist frequency exists
        magnitudes[-1] /= 2.0
    
    phases = np.angle(fft_values)
    
    # Create figures:
    if fig1 is None:
        fig1 = plt.figure(figsize=(10, 6))
        plt.plot(freqs_per_year, magnitudes, linewidth=1, label=name)
        plt.title('FFT Magnitude Spectrum (Linear)', fontsize=16)
        plt.xlabel('Frequency (1/yr)', fontsize=14)
        plt.ylabel('Magnitude', fontsize=14)
        plt.grid(True, which='major', alpha=0.5)
        plt.grid(True, which='minor', alpha=0.5)
        plt.minorticks_on()
        plt.xticks(fontsize=14)
        plt.yticks(fontsize=14)
        plt.tight_layout()
        if name:
            plt.legend(loc='upper right')
    else:
        plt.figure(fig1.number)
        plt.plot(freqs_per_year, magnitudes, linewidth=1, label=name)
        if name:
            plt.legend(loc='upper right')
    
    if fig2 is None:
        fig2 = plt.figure(figsize=(10, 6))
        plt.loglog(freqs_per_year[1:], magnitudes[1:], linewidth=1, label=name)
        plt.title('FFT Magnitude Spectrum (log)', fontsize=16)
        plt.xlabel('Frequency (1/yr)', fontsize=14)
        plt.ylabel('Magnitude', fontsize=14)
        plt.xticks(fontsize=14)
        plt.yticks(fontsize=14)
        plt.grid(True, which='major', alpha=0.5)
        plt.grid(True, which='minor', alpha=0.5)
        #plt.axvline(x=3.5, color='green', label='3.5 cycles/yr', linestyle='--')
        #plt.axvline(x=182.625, color='red', label='182.625 cycles/yr', linestyle='--')
        #plt.axvline(x=1.826, color='green', label='1.826 cycles/yr', linestyle='--')
        #plt.axvline(x=0.5479, color='red', label='0.5479 cycles/yr', linestyle='--')
        plt.tight_layout()
        if name:
            plt.legend(loc='upper right')
    else:
        plt.figure(fig2.number)
        plt.loglog(freqs_per_year[1:], magnitudes[1:], linewidth=1, label=name)
        plt.legend()
        if name:
            plt.legend(loc='upper right')
    
    if fig3 is None:
        fig3 = plt.figure(figsize=(10, 6))
        plt.plot(freqs_per_year[1:], phases[1:], linewidth=1, label=name)
        plt.title('Phase Spectrum', fontsize=16)
        plt.xlabel('Frequency (1/yr)', fontsize=14)
        plt.ylabel('Phase (radians)', fontsize=14)
        plt.grid(True, alpha=0.5)
        plt.xticks(fontsize=14)
        plt.yticks(fontsize=14)
        #plt.xlim(-0.1, 5)
        plt.tight_layout()
        if name:
            plt.legend(loc='lower right', framealpha=0.5)
    else:
        plt.figure(fig3.number)
        plt.plot(freqs_per_year[1:], phases[1:], linewidth=1, label=name)
        if name:
            plt.legend(loc='lower right', framealpha=0.8)
    
    return fig1, fig2, fig3


# Example usage:
# Assuming you have your dataframe df with column 'Normalized_Shifted'

decimation_factor = 52  # step for subsampling
sr = 1  # Original sampling rate: 1 sample/day

# For full data
fig1, fig2, fig3 = plotly_fft_2(df1['Normalized_Shifted'].values, sr, name='Full Data', apply_window=False)

# For subsampled data with anti-aliasing
tapered_sub1, new_sr = subsample_with_antialiasing(df1['Normalized_Shifted'].values, decimation_factor, 8, sr)
plotly_fft_2(tapered_sub1, new_sr, fig1=fig1, fig2=fig2, fig3=fig3, name='Subsampled/Tapered Data', apply_window=True)

# Print sampling information
print(f"Original sampling rate: {sr} sample/day")
print(f"Original Nyquist: {sr/2} cycle/day")
print(f"Original Nyquist in cycles/year {sr/2 * 365.25} cycle/year")
print(f"Decimation factor: {decimation_factor}")
new_sr_example = sr / decimation_factor
print(f"New sampling rate: {new_sr_example} sample/day")
print(f"New Nyquist: {new_sr_example/2} cycle/day")
print(f"New Nyquist in cycles/year: {new_sr_example/2 * 365.25} cycle/year")

# plt.show()

***
### Adding Sine Waves to Full DNS Data
#### Notes:
- two data columns containing sine waves are added to the data frame containing the full DNS data (not tapered or subsampled yet), along with additional columns for the combined sine waves, the time index, and the DNS data combined with both sine waves
- the first year of DNS data is also plotted alongside the individual sine waves before combining

In [ ]:
df2 = df1['Normalized_Shifted'].to_frame()

In [ ]:
# Add sine waves to df2 data columns
def add_sine_waves(data, data_to_add_sines_to=''):
    data = data.copy()
    
    data['time_column'] = np.arange(len(data))
    t = data['time_column'].values # time is in days and so is the data collection, so index matches day value
    
    # Define sine wave parameters
    frequencies = {'sine_0_05_cpd': 0.005,  # cpd: cycles per day
                'sine_0_15_cpd': 0.0015}
    
    amplitudes = {'sine_0_05_cpd': 0.25,
                'sine_0_15_cpd': 0.25}
    
    phases = {'sine_0_05_cpd': 0,
           'sine_0_15_cpd': 0}
    
    # Add sine wave columns
    for wave_name, freq in frequencies.items():
        amplitude = amplitudes[wave_name]
        phase = phases[wave_name]
        
        # Calculate sine wave: A * sin(2π * f * t + φ)
        data[wave_name] = amplitude * np.sin(2 * np.pi * freq * t + phase)
        
    # Add combined sine waves column
    sine_columns = list(frequencies.keys())
    data['combined_sines'] = data[sine_columns].sum(axis=1)

    # Add sine waves to original data        
    if data_to_add_sines_to in data.columns:
        data['Data_With_Sines'] = data[data_to_add_sines_to] + data['combined_sines']

    if 'time_column' in data.index:
        data = data.drop('time_column', axis=0) # prevents time_column from appearing as an additional row

    return data

add_sine_waves(df2, data_to_add_sines_to='Normalized_Shifted')

In [ ]:
add_sine_waves(df2)
print(df2.columns.tolist())  # Check the exact column names

In [ ]:
# Plot time series with separate sine waves:
df2_with_sines = add_sine_waves(df2, data_to_add_sines_to='Normalized_Shifted') # need new variable for edited dataframe

plt.figure(figsize=(12,6))
plt.plot(df2_with_sines['time_column'][:365], df2_with_sines['Normalized_Shifted'][:365], label='Normalized time series')
plt.plot(df2_with_sines['time_column'][:365], df2_with_sines['sine_0_05_cpd'][:365], label='sine_0_005_cpd')
plt.plot(df2_with_sines['time_column'][:365], df2_with_sines['sine_0_15_cpd'][:365], label='sine_0_0015_cpd', linestyle=":")
plt.xlabel("Time (days)")
plt.ylabel("Amplitude")
plt.title("Sine Waves Before Adding to Data (First Year)")
plt.grid(True, alpha=0.5)
plt.tight_layout()
plt.legend()
plt.show()

# Trying with plotly:
pre_sines = go.Figure()
pre_sines.add_trace(go.Scatter(x= list(df1.index), y= list(df2_with_sines['Normalized_Shifted']), mode='lines', name='Time Series'))
pre_sines.add_trace(go.Scatter(x= list(df1.index), y= list(df2_with_sines['sine_0_05_cpd']), mode='lines', name='0.005 cpd sine'))
pre_sines.add_trace(go.Scatter(x= list(df1.index), y= list(df2_with_sines['sine_0_15_cpd']), mode='lines', name='0.0015 cpd sine'))
# Set title
pre_sines.update_layout(title_text="Before Adding Sine Waves to Water Level Data (1960-2024)", width=1100, height=600, 
                  xaxis_title='Date', yaxis_title='Water Level', title_x=0.5, showlegend=True)
# Add range slider
pre_sines.update_layout(xaxis=dict(rangeslider=dict(visible=True)))
pre_sines.show()
# Zoom in to see more than blob

***
### Adding Sine Waves to Subsampled/Tapered Time Series
#### Notes:
- using add_sine_waves function from previous cell

In [ ]:
add_sine_waves(tapered_subsampled, data_to_add_sines_to='Subsampled_Tapered')

In [ ]:
sub_t_with_sines = add_sine_waves(tapered_subsampled, data_to_add_sines_to='Subsampled_Tapered')

# Plotting time series with sines before combining:
pre_2sines = go.Figure()
pre_2sines.add_trace(go.Scatter(x= list(tapered_subsampled['date_column']), y= list(sub_t_with_sines['Subsampled_Tapered']), mode='lines', 
                                name='Tapered Time Series (step = 52)', zorder=10))
pre_2sines.add_trace(go.Scatter(x= list(tapered_subsampled['date_column']), y= list(sub_t_with_sines['sine_0_05_cpd']), mode='lines', 
                                name='0.005 cpd sine', line=dict(dash='solid')))
pre_2sines.add_trace(go.Scatter(x= list(tapered_subsampled['date_column']), y= list(sub_t_with_sines['sine_0_15_cpd']), mode='lines', 
                                name='0.0015 cpd sine', line=dict(dash='solid')))
# Set title
pre_2sines.update_layout(title_text="Subsampled and Tapered Water Level Data Before Adding Sine Waves (1960-2024)", width=1100, height=600, 
                  xaxis_title='Date', yaxis_title='Water Level', title_x=0.5, showlegend=True)
# Add range slider
pre_2sines.update_layout(xaxis=dict(rangeslider=dict(visible=True)))
pre_2sines.show()

***
### Fast Fourier Transform for Subsampled and Tapered DNS Data with Added Sine Waves
#### Notes:
- Calling plotly_fft2 function from a previous cell
- Sine spikes should appear at 1.826 cycles/yr and 0.5479 cycles/year

In [ ]:
# Trying to fix display of above sine FFT plots same way as above
decimation_factor = 52  # step for subsampling
sr = 1  # Original sampling rate: 1 sample/day

# For full data
fig1, fig2, fig3 = plotly_fft_2(df2_with_sines['Data_With_Sines'].values, sr, name='Full Data', apply_window=False)

# For subsampled data with anti-aliasing
tapered_sub1, new_sr = subsample_with_antialiasing(df2_with_sines['Data_With_Sines'].values, decimation_factor, 8, sr) 
plotly_fft_2(tapered_sub1, new_sr, fig1=fig1, fig2=fig2, fig3=fig3, name='Subsampled/Tapered Data', apply_window=True)

plt.show()

In [ ]:
freq1 = 0.0015  # cycles/day
freq2 = 0.005   # cycles/day

period1 = 1/freq1  # days
period2 = 1/freq2  # days

samples_per_cycle1 = period1 / 52
samples_per_cycle2 = period2 / 52

print(f"Sine wave 1: {samples_per_cycle1:.2f} samples per cycle")
print(f"Sine wave 2: {samples_per_cycle2:.2f} samples per cycle")

***
### Continuous Wavelet Transform (Full DNS Data)
#### Notes:
- the Morlet wavelet, given by $\psi(t)=\exp(\frac{-t^2}{2})\cos(5t)$, is used for the wavelet transform
- scalograms are created using the full DNS data and the full DNS data combined with sine waves
- histograms of scalogram magnitudes are also plotted showing the percentages corresponding to certain magnitude ranges in addition to other statistics (i.e. mean, median, mode, etc.)
- a difference map was also created to compare scalograms

***
### Increasing Scalogram Frequency Resolution
#### Notes:
- **goal**: increase frequency resolution of scalograms
  
    **Consequences of increasing frequency resolution:**
    - increasing the frequency resolution will decrease the time resolution
    - will change appearance of added sine waves
      
    <br> **Possible Methods for increasing frequency resolution:**
    - Increasing the number of scales will provide more frequency bins to sample, which will show finer differences between frequencies
    - Adjust spacing to linear instead of logarithmic and focus on a specific frequency range of interest
    - Decrease time resolution
      <br>**Effective Methods:**
      - Increasing the number of scales showed a slight difference compared to the starting scalogram
      - Decreasing time resolution by applying a uniform filter from scipy.ndimage had the greatest effect on the appearance of the plots
          - The filter is set up to only smooth in time and leave frequency unchanged
<br>

- Scales have been determined based on the formula $f=\frac{F_c}{a\cdot\Delta t}$, where $f$ is the pseudo-frequency corresponding to the scale, $F_c$ is the approx. center frequency of the wavelet, $a$ is the wavelet scale, and $\Delta t$ is the sampling period of the signal
- Additionally, scales are chosen so that the frequency range of the plots do not exceed the Nyquist frequency

#### Full DNS Data Without Added Sine Waves

In [ ]:
# continuous wavelet transform without sine wave added to data
# Convert scale to frequency:
data = df2['Normalized_Shifted'].values 
data = data.flatten() 
#scales = np.logspace(np.log10(1), np.log10(100), 120)

t = df2_with_sines['time_column'].values
dt = t[1] - t[0]
print(f"dt: {dt} day")
#scale_min = 1.0 / (10 * dt)  # Small scale for high freq, use freq_max=10
freq_max_years = 150 # below Nyquist limit of ~182.6
scale_min = 1.0/(freq_max_years/ 365.25)
scale_max = 1.0 / (0.001 * dt)  # Large scale for low freq, use freq_min=0.001
scales = np.logspace(np.log10(scale_min), np.log10(scale_max), 300)
coefficients, frequencies = pywt.cwt(data, scales, 'morl')
s2f_per_year = frequencies*365.25

# Smooth only in the TIME direction (axis=1), not frequency (axis=0)
smoothed_coefficients = uniform_filter(np.abs(coefficients), size=(1, 20))

plt.figure(figsize=(14, 6))
# try log scale:
plt.pcolormesh(t, s2f_per_year, np.log10(smoothed_coefficients), cmap='plasma', shading='auto', vmin=-5)
plt.colorbar(label='log\u2081\u2080(Magnitude)')
plt.ylabel('Frequency (1/Year)')
plt.yscale('log')
# Convert time to years
years = np.arange(1960, 2025, 10)
year_days = (years - 1960) * 365.25
plt.xticks(year_days, years)
plt.xlabel('Year')
plt.title('Toronto Daily Mean Water Level Without Added Sine Waves (1960-2024)') # from normalized + detrended + mean-shifted water level data
plt.show()
print(f"Frequency range: {s2f_per_year.min():.3f} to {s2f_per_year.max():.3f} cycles/year")

# Add histogram:
magnitudes = np.abs(smoothed_coefficients)
magnitude_values = magnitudes.flatten()
# Then create histogram
fig_hist = go.Figure()

fig_hist.add_trace(go.Histogram(x=magnitude_values, nbinsx=50,  # Number of bins
        histnorm='percent',  # Options: '', 'percent', 'probability', 'density', 'probability density'
        name='Scalogram Magnitudes', marker=dict(opacity=0.7, line=dict(color='darkblue', width=1))))

# Add mean magnitude line:
mean_mags = np.mean(magnitude_values)
fig_hist.add_vline( x=mean_mags, line_dash="dash", line_color="red")
# Add median magnitude line:
median_mags = np.median(magnitude_values)
fig_hist.add_vline(x=median_mags, line_dash="dash", line_color="orange")
# Get mode:
# Find bin with maximum count
hist_counts, bin_edges = np.histogram(magnitude_values, bins=50)
mode_bin_index = np.argmax(hist_counts)
# Mode is the center of the bin with the highest count:
mode_value = (bin_edges[mode_bin_index] + bin_edges[mode_bin_index + 1]) / 2
fig_hist.add_vline(x=mode_value, line_dash="dash", line_color="limegreen")

# Add traces for legend entries
fig_hist.add_trace(go.Scatter(x=[None], y=[None], mode='lines', line=dict(color='red', dash='dash'),name=f'Mean: {mean_mags:.2f}', showlegend=True))

fig_hist.add_trace(go.Scatter(x=[None], y=[None], mode='lines',line=dict(color='orange', dash='dash'),name=f'Median: {median_mags:.2f}',showlegend=True))

fig_hist.add_trace(go.Scatter(x=[None], y=[None], mode='lines',line=dict(color='limegreen', dash='dash'),name=f'Mode: {mode_value:.2f}', showlegend=True))

fig_hist.update_layout(title='Histogram of Scalogram Magnitudes', xaxis=dict(title='Magnitude'), yaxis=dict(title='Percentage'),
        template='plotly_white', width=1000, height=600, showlegend=True)

# Log scale for large span of magnitudes
fig_hist.update_yaxes(type="log")

fig_hist.show()

print(ss.describe(magnitude_values))
std_dev = np.std(magnitude_values)
print(f"Standard Deviation: {std_dev:.4f}")

#### Full DNS Data With Added Sine Waves

In [ ]:
# Continuous wavelet transform with sine waves added
data1 = df2_with_sines['Data_With_Sines'].values  # Normalized data, Convert to numpy array
data1 = data1.flatten()  # Need data to be 1D
print(f"Data shape: {data1.shape}") # Check data shape
t = df2_with_sines['time_column'].values
dt = t[1] - t[0]
print(f"dt: {dt} day")
#scale_min = 1.0 / (10 * dt)  # Small scale for high freq, use freq_max=10
freq_max_years = 150 # below Nyquist limit of ~182.6
scale_min = 1.0/(freq_max_years/ 365.25)
scale_max = 1.0 / (0.001 * dt)  # Large scale for low freq, use freq_min=0.001
scales1 = np.logspace(np.log10(scale_min), np.log10(scale_max), 300)
# plot with years instead of days
t_years = t / 365.25  # Convert days to years
# CWT:
coefficients1, frequencies1 = pywt.cwt(data1, scales1, 'morl', sampling_period=1.0)
print(f"Coefficients shape: {coefficients1.shape}")
# Convert frequencies from cycles/day to cycles/year
s2f_per_year1 = frequencies1*365.25

# Trying to apply uniform_filter from scipy.ndimage to decrease time resolution and make frequency content easier to see
# Smooth only in the TIME direction (axis=1), not frequency (axis=0)
coefficients_smoothed = uniform_filter(np.abs(coefficients1), size=(1, 20))
# size=(rows, columns) or size=(frequency_axis, time_axis)
# for size=(1, 10): No smoothing in frequency, smooth over 10 time points in time

plt.figure(figsize=(14, 6))
plt.pcolormesh(t_years, s2f_per_year1, (coefficients_smoothed), cmap='plasma', shading='auto', vmax=8) 
plt.colorbar(label='Magnitude')
#plt.colorbar(label='log\u2081\u2080(Magnitude)')
plt.ylabel('Frequency (1/year)')
plt.yscale('log')
plt.xlabel('Time (years)')
plt.title('Toronto Daily Mean Water Level With Added Sine Waves (1960-2024)') # from normalized + detrended + mean-shifted water level data
plt.show()

# Add histogram:
magnitudes = np.abs(coefficients_smoothed)
magnitude_values = magnitudes.flatten()
# Then create histogram
fig_hist = go.Figure()

fig_hist.add_trace(go.Histogram(x=magnitude_values, nbinsx=50,  # Number of bins
        histnorm='percent',  # Options: '', 'percent', 'probability', 'density', 'probability density'
        name='Scalogram Magnitudes', marker=dict(opacity=0.7, line=dict(color='darkblue', width=1))))

# Add mean magnitude line:
mean_mags = np.mean(magnitude_values)
fig_hist.add_vline( x=mean_mags, line_dash="dash", line_color="red")
# Add median magnitude line:
median_mags = np.median(magnitude_values)
fig_hist.add_vline(x=median_mags, line_dash="dash", line_color="orange")
# Get mode:
# Find bin with maximum count
hist_counts, bin_edges = np.histogram(magnitude_values, bins=50)
mode_bin_index = np.argmax(hist_counts)
# Mode is the center of the bin with the highest count:
mode_value = (bin_edges[mode_bin_index] + bin_edges[mode_bin_index + 1]) / 2
fig_hist.add_vline(x=mode_value, line_dash="dash", line_color="limegreen")

# Add traces for legend entries
fig_hist.add_trace(go.Scatter(x=[None], y=[None], mode='lines', line=dict(color='red', dash='dash'),name=f'Mean: {mean_mags:.2f}', showlegend=True))

fig_hist.add_trace(go.Scatter(x=[None], y=[None], mode='lines',line=dict(color='orange', dash='dash'),name=f'Median: {median_mags:.2f}',showlegend=True))

fig_hist.add_trace(go.Scatter(x=[None], y=[None], mode='lines',line=dict(color='limegreen', dash='dash'),name=f'Mode: {mode_value:.2f}', showlegend=True))

fig_hist.update_layout(title='Histogram of Scalogram Magnitudes', xaxis=dict(title='Magnitude'), yaxis=dict(title='Percentage'),
        template='plotly_white', width=1000, height=600, showlegend=True)

# Log scale for large span of magnitudes
fig_hist.update_yaxes(type="log")

fig_hist.show()

print(ss.describe(magnitude_values))
std_dev = np.std(magnitude_values)
print(f"Standard Deviation: {std_dev:.4f}")

In [ ]:
# Difference map:
# trying to make difference map for smoothed subsampled/tapered data
signal_original = df2['Normalized_Shifted'].values.flatten() # WITHOUT sine waves

signal_with_sine = df2_with_sines['Data_With_Sines'].values.flatten()  # WITH sine waves

# Get the time indices from subsampled data
t = df2_with_sines['time_column'].values
# Account for the step size of 52 
t_years = 1960 + (t / 365.25)  # Convert days to years)
dt = t[1] - t[0]
print(f"dt: {dt} day")
#scale_min = 1.0 / (10 * dt)  # Small scale for high freq, use freq_max=10
freq_max_years = 150 # below Nyquist limit of ~182.6
scale_min = 1.0/(freq_max_years/ 365.25)
scale_max = 1.0 / (0.001 * dt)  # Large scale for low freq, use freq_min=0.001
scales1 = np.logspace(np.log10(scale_min), np.log10(scale_max), 300)

coefficients_original, freqs = pywt.cwt(signal_original, scales, 'morl')
coefficients_with_sine, freqs = pywt.cwt(signal_with_sine, scales, 'morl')
s2f_per_year = freqs*365.25

coefficients_smoothed = uniform_filter(np.abs(coefficients_original), size=(1, 20))
sines_coefficients_smoothed = uniform_filter(np.abs(coefficients_with_sine), size=(1,20))

fig, axes = plt.subplots(1, 3, figsize=(15, 4))
# Scalogram 1 - use log scale like original
im1 = axes[0].pcolormesh(t_years, s2f_per_year, np.log10(coefficients_smoothed), cmap='plasma', shading='auto', vmin=-5)
axes[0].set_title('Scalogram 1')
axes[0].set_xlabel('Year')  # or 'Time (years)'
axes[0].set_ylabel('Frequency (1/year)')
axes[0].set_yscale('log')  # Use log scale for frequency axis
plt.colorbar(im1, ax=axes[0], label='log\u2081\u2080(Magnitude)')

# Scalogram 2
im2 = axes[1].pcolormesh(t_years, s2f_per_year, (sines_coefficients_smoothed), cmap='plasma', shading='auto', vmax=8) 
axes[1].set_title('Scalogram 2')
axes[1].set_xlabel('Year')
axes[1].set_ylabel('Frequency (1/year)')
axes[1].set_yscale('log')
plt.colorbar(im2, ax=axes[1], label='Magnitude')

# Normalize each scalogram
coeff_norm = (np.abs(coefficients_smoothed) - np.abs(coefficients_smoothed).min()) / (np.abs(coefficients_smoothed).max() - np.abs(coefficients_smoothed).min())
coeff1_norm = (np.abs(sines_coefficients_smoothed) - np.abs(sines_coefficients_smoothed).min()) / (np.abs(sines_coefficients_smoothed).max() - np.abs(sines_coefficients_smoothed).min())

difference = coeff_norm - coeff1_norm
# Difference map
max_diff = np.max(np.abs(difference))
im3 = axes[2].pcolormesh(t_years, s2f_per_year, difference, cmap='RdBu', vmin=-max_diff, vmax=max_diff, shading='auto')
axes[2].set_title('Difference (1 - 2)')
axes[2].set_xlabel('Year')
axes[2].set_ylabel('Frequency (1/year)')
axes[2].set_yscale('log')
plt.colorbar(im3, ax=axes[2], label='Difference')

print("Comparing scalograms for full DNS data with and without added sine waves:")

plt.tight_layout()
plt.show()

***
### Continuous Wavelet Transform (Subsampled/Tapered DNS Data)
#### Notes:
- The Morlet wavelet, given by $\psi(t)=\exp(\frac{-t^2}{2})\cos(5t)$, is used for the wavelet transform
- Again, histograms and difference maps were created for the scalograms

#### Subsampled and Tapered DNS Data Without Added Sine Waves

In [ ]:
sr=1
def apply_hann_window(data):
    # Want to apply Hann window and return windowed data with coherent gain factor
    window = np.hanning(len(data))
    windowed_data = data * window
    # Coherent gain for Hann window is 0.5, so need to multiply by 2 to make correction
    coherent_gain = 0.5
    return windowed_data/coherent_gain

def subsample_with_antialiasing(data, decimation_factor, sr):
    """Subsample data using scipy.signal.decimate with anti-aliasing filter.
    data: array
    decimation_factor: factor by which to reduce sampling rate
    sr: Original sampling rate"""
    
    # scipy.signal.decimate applies an anti-aliasing filter automatically
    # Use a higher order filter for better anti-aliasing (default is 8)
    decimated_data = scipy.signal.decimate(data, decimation_factor, ftype='iir', zero_phase=True)
    # get new sampling rate:
    new_sr = sr / decimation_factor
    return decimated_data

hann_windowed_data = apply_hann_window(df2['Normalized_Shifted'].values)
hann_windowed_sines = apply_hann_window(df2_with_sines['Data_With_Sines'].values)
subsample_with_antialiasing(hann_windowed_data, 52, sr)
subsample_with_antialiasing(hann_windowed_sines, 52, sr)
# doesn't make noticable difference for scalograms below.

In [ ]:
## CHECK (make sure sub/tap scalograms are showing up correctly)
sub_t_data = sub_t_with_sines['Subsampled_Tapered'].values.flatten()

# Get the time indices from subsampled data
t = sub_t_with_sines['time_column'].values
# Account for the step size of 52 -> IMPORTANT
sampling_interval_days = 52  # subsampled every 52nd day
t_days = t * sampling_interval_days  # Convert indices to actual days
t_years = 1960 + (t_days / 365.25)  # Convert days to years
dt = t_days[1] - t_days[0]
print(f"dt: {dt} day")

scale_min = 1.0 / (0.01 * dt)  # Small scale for high freq, use freq_max=0.01
scale_max = 1.0 / (0.00001 * dt)  # Large scale for low freq, use freq_min=0.00001
# note: trying to use same scales for full and subsampled/tapered data is not working
scales = np.logspace(np.log10(scale_min), np.log10(scale_max), 300)

coefficients, frequencies = pywt.cwt(sub_t_data, scales, 'morl')
s2f_per_year = frequencies*365.25

# smooth in time
subt_coefficients_smoothed = uniform_filter(np.abs(coefficients), size=(1, 5))

plt.figure(figsize=(14, 6))
# try log scale:
plt.pcolormesh(t_years, s2f_per_year, np.log10(subt_coefficients_smoothed), cmap='plasma', shading='auto', vmin=-5)
plt.colorbar(label='log\u2081\u2080(Magnitude)')
plt.ylabel('Frequency (1/Year)')
plt.yscale('log')
# Convert time to years
years = np.arange(1960, 2025, 10)
plt.xticks(years)
plt.xlabel('Year')
plt.title('Subsampled and Tapered Toronto Daily Mean Water Level Data Without Added Sine Waves (1960-2024)')
plt.show()
print(f"Frequency range: {s2f_per_year.min():.3f} to {s2f_per_year.max():.3f} cycles/year")

# Add histogram:
magnitudes = np.abs(subt_coefficients_smoothed)
magnitude_values = magnitudes.flatten()
# Then create histogram
fig_hist = go.Figure()

fig_hist.add_trace(go.Histogram(x=magnitude_values, nbinsx=50,  # Number of bins
        histnorm='percent',  # Options: '', 'percent', 'probability', 'density', 'probability density'
        name='Scalogram Magnitudes', marker=dict(opacity=0.7, line=dict(color='darkblue', width=1))))

# Add mean magnitude line:
mean_mags = np.mean(magnitude_values)
fig_hist.add_vline( x=mean_mags, line_dash="dash", line_color="red")
# Add median magnitude line:
median_mags = np.median(magnitude_values)
fig_hist.add_vline(x=median_mags, line_dash="dash", line_color="orange")
# Get mode:
# Find bin with maximum count
hist_counts, bin_edges = np.histogram(magnitude_values, bins=50)
mode_bin_index = np.argmax(hist_counts)
# Mode is the center of the bin with the highest count:
mode_value = (bin_edges[mode_bin_index] + bin_edges[mode_bin_index + 1]) / 2
fig_hist.add_vline(x=mode_value, line_dash="dash", line_color="limegreen")

# Add traces for legend entries
fig_hist.add_trace(go.Scatter(x=[None], y=[None], mode='lines', line=dict(color='red', dash='dash'),name=f'Mean: {mean_mags:.2f}', showlegend=True))

fig_hist.add_trace(go.Scatter(x=[None], y=[None], mode='lines',line=dict(color='orange', dash='dash'),name=f'Median: {median_mags:.2f}',showlegend=True))

fig_hist.add_trace(go.Scatter(x=[None], y=[None], mode='lines',line=dict(color='limegreen', dash='dash'),name=f'Mode: {mode_value:.2f}', showlegend=True))

fig_hist.update_layout(title='Histogram of Scalogram Magnitudes', xaxis=dict(title='Magnitude'), yaxis=dict(title='Percentage'),
        template='plotly_white', width=1000, height=600, showlegend=True)

# Log scale for large span of magnitudes
fig_hist.update_yaxes(type="log")

fig_hist.show()

print(ss.describe(magnitude_values))
std_dev = np.std(magnitude_values)
print(f"Standard Deviation: {std_dev:.4f}")

#### Subsampled and Tapered Data With Added Sine Waves

In [ ]:
sub_t_sines_data = sub_t_with_sines['Data_With_Sines'].values.flatten() 

# Get the time indices from subsampled data
t = sub_t_with_sines['time_column'].values
# Account for the step size of 52 
sampling_interval_days = 52  # subsampled every 52nd day
t_days = t * sampling_interval_days  # Convert indices to actual days
t_years = 1960 + (t_days / 365.25)  # Convert days to years)
dt = t_days[1] - t_days[0]
print(f"dt: {dt} day")

scale_min = 1.0 / (0.01 * dt)  # Small scale for high freq, use freq_max=0.01
scale_max = 1.0 / (0.00001 * dt)  # Large scale for low freq, use freq_min=0.00001
scales = np.logspace(np.log10(scale_min), np.log10(scale_max), 300)

coefficients, frequencies = pywt.cwt(sub_t_sines_data, scales, 'morl')
s2f_per_year = frequencies*365.25

# smooth in time
tsub_coefficients_smoothed = uniform_filter(np.abs(coefficients), size=(1, 10))

plt.figure(figsize=(14, 6))
# try log scale:
#plt.pcolormesh(t_years, s2f_per_year, np.log10(tsub_coefficients_smoothed), cmap='plasma', shading='auto', vmin=-5)
#plt.colorbar(label='log\u2081\u2080(Magnitude)')
plt.pcolormesh(t_years, s2f_per_year, tsub_coefficients_smoothed, cmap='plasma', shading='auto', vmin=-0.2)
plt.colorbar(label='Magnitude')
plt.ylabel('Frequency (1/Year)')
plt.yscale('log')
#plt.axhline(y=0.5479, color='white', linestyle='--', label='Sine 2')
#plt.axhline(y=1.826, color='skyblue', linestyle='--', label='Sine 1')
# Convert time to years
years = np.arange(1960, 2025, 10)
plt.xticks(years)
plt.xlabel('Year')
plt.title('Subsampled and Tapered Toronto Daily Mean Water Level Data With Added Sine Waves (1960-2024)')
plt.show()
print(f"Frequency range: {s2f_per_year.min():.3f} to {s2f_per_year.max():.3f} cycles/year")

# Add histogram:
magnitudes = np.abs(tsub_coefficients_smoothed)
magnitude_values = magnitudes.flatten()
# Then create histogram
fig_hist = go.Figure()

fig_hist.add_trace(go.Histogram(x=magnitude_values, nbinsx=50,  # Number of bins
        histnorm='percent',  # Options: '', 'percent', 'probability', 'density', 'probability density'
        name='Scalogram Magnitudes', marker=dict(opacity=0.7, line=dict(color='darkblue', width=1))))

# Add mean magnitude line:
mean_mags = np.mean(magnitude_values)
fig_hist.add_vline( x=mean_mags, line_dash="dash", line_color="red")
# Add median magnitude line:
median_mags = np.median(magnitude_values)
fig_hist.add_vline(x=median_mags, line_dash="dash", line_color="orange")
# Get mode:
# Find bin with maximum count
hist_counts, bin_edges = np.histogram(magnitude_values, bins=50)
mode_bin_index = np.argmax(hist_counts)
# Mode is the center of the bin with the highest count:
mode_value = (bin_edges[mode_bin_index] + bin_edges[mode_bin_index + 1]) / 2
fig_hist.add_vline(x=mode_value, line_dash="dash", line_color="limegreen")

# Add traces for legend entries
fig_hist.add_trace(go.Scatter(x=[None], y=[None], mode='lines', line=dict(color='red', dash='dash'),name=f'Mean: {mean_mags:.2f}', showlegend=True))

fig_hist.add_trace(go.Scatter(x=[None], y=[None], mode='lines',line=dict(color='orange', dash='dash'),name=f'Median: {median_mags:.2f}',showlegend=True))

fig_hist.add_trace(go.Scatter(x=[None], y=[None], mode='lines',line=dict(color='limegreen', dash='dash'),name=f'Mode: {mode_value:.2f}', showlegend=True))

fig_hist.update_layout(title='Histogram of Scalogram Magnitudes', xaxis=dict(title='Magnitude'), yaxis=dict(title='Percentage'),
        template='plotly_white', width=1000, height=600, showlegend=True)

# Log scale for large span of magnitudes
fig_hist.update_yaxes(type="log")

fig_hist.show()

print(ss.describe(magnitude_values))
std_dev = np.std(magnitude_values)
print(f"Standard Deviation: {std_dev:.4f}")

In [ ]:
# trying to make difference map for smoothed subsampled/tapered data
st_signal_original = sub_t_with_sines['Subsampled_Tapered'].values.flatten() # WITHOUT sine waves

st_signal_with_sine = sub_t_with_sines['Data_With_Sines'].values.flatten()  # WITH sine waves

# Get the time indices from subsampled data
t = sub_t_with_sines['time_column'].values
# Account for the step size of 52 
sampling_interval_days = 52  # subsampled every 52nd day
t_days = t * sampling_interval_days  # Convert indices to actual days
t_years = 1960 + (t_days / 365.25)  # Convert days to years)
dt = t_days[1] - t_days[0]
print(f"dt: {dt} day")

scale_min = 1.0 / (0.01 * dt)  # Small scale for high freq, use freq_max=0.01
scale_max = 1.0 / (0.00001 * dt)  # Large scale for low freq, use freq_min=0.00001
scales = np.logspace(np.log10(scale_min), np.log10(scale_max), 300)

coefficients_original_st, freqs = pywt.cwt(st_signal_original, scales, 'morl')
coefficients_with_sine_st, freqs = pywt.cwt(st_signal_with_sine, scales, 'morl')
s2f_per_year = freqs*365.25

tsub_coefficients_smoothed = uniform_filter(np.abs(coefficients_original_st), size=(1, 10))
tsub_sines_coefficients_smoothed = uniform_filter(np.abs(coefficients_with_sine_st), size=(1,10))

fig, axes = plt.subplots(1, 3, figsize=(15, 4))
# Scalogram 1 - use log scale like original
im1 = axes[0].pcolormesh(t_years, s2f_per_year, np.log10(tsub_coefficients_smoothed), cmap='plasma', shading='auto', vmin=-5)
axes[0].set_title('Scalogram 1')
axes[0].set_xlabel('Year')  # or 'Time (years)'
axes[0].set_ylabel('Frequency (1/year)')
axes[0].set_yscale('log')  # Use log scale for frequency axis
plt.colorbar(im1, ax=axes[0], label='log\u2081\u2080(Magnitude)')

# Scalogram 2
im2 = axes[1].pcolormesh(t_years, s2f_per_year, tsub_sines_coefficients_smoothed, cmap='plasma', shading='auto')
axes[1].set_title('Scalogram 2')
axes[1].set_xlabel('Year')
axes[1].set_ylabel('Frequency (1/year)')
axes[1].set_yscale('log')
plt.colorbar(im2, ax=axes[1], label='Magnitude')

# Normalize each scalogram
coeff_norm = (np.abs(tsub_coefficients_smoothed) - np.abs(tsub_coefficients_smoothed).min()) / (np.abs(tsub_coefficients_smoothed).max() - np.abs(tsub_coefficients_smoothed).min())
coeff1_norm = (np.abs(tsub_sines_coefficients_smoothed) - np.abs(tsub_sines_coefficients_smoothed).min()) / (np.abs(tsub_sines_coefficients_smoothed).max() - np.abs(tsub_sines_coefficients_smoothed).min())

difference = coeff_norm - coeff1_norm
# Difference map
max_diff = np.max(np.abs(difference))
im3 = axes[2].pcolormesh(t_years, s2f_per_year, difference, cmap='RdBu', vmin=-max_diff, vmax=max_diff, shading='auto')
axes[2].set_title('Difference (1 - 2)')
axes[2].set_xlabel('Year')
axes[2].set_ylabel('Frequency (1/year)')
axes[2].set_yscale('log')
plt.colorbar(im3, ax=axes[2], label='Difference')

print("Comparing scalograms for subsampled/tapered data with and without added sine waves:")

plt.tight_layout()
plt.show()

#### Possible issues with scalograms:
- Horizontal banding near bottom of plots, in particular in the subsampled/tapered scalogram without sine waves added